# Setup

In [1]:
import tensorflow as tf
import numpy as np
import os
import time

tf.enable_eager_execution()

## Download Shakespeare dataset

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

## Read the data

In [3]:
text = open(path_to_file).read()
print("Length of text: {} characters".format(len(text)))
print(text[:250])
vocab = sorted(set(text))
print("{} unique characters".format(len(vocab)))

Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

65 unique characters


# Process the text

## Vectorize the text

In [4]:
# Mapping from unique characters to indices
char2idx = { uniq_ch: i for i, uniq_ch in enumerate(vocab)}
idx2char = np.array(vocab)
print(char2idx['c'])
print(idx2char[41])

text_as_int = np.array([char2idx[c] for c in text])
print(text_as_int[:10])
print(text[:10])

41
c
[18 47 56 57 58  1 15 47 58 47]
First Citi


In [5]:
print("{")
for char, _ in zip(char2idx, range(20)):
    print("    {:4s}: {:3d},".format(repr(char), char2idx[char]))
print("    ...\n}")

{
    '\n':   0,
    ' ' :   1,
    '!' :   2,
    '$' :   3,
    '&' :   4,
    "'" :   5,
    ',' :   6,
    '-' :   7,
    '.' :   8,
    '3' :   9,
    ':' :  10,
    ';' :  11,
    '?' :  12,
    'A' :  13,
    'B' :  14,
    'C' :  15,
    'D' :  16,
    'E' :  17,
    'F' :  18,
    'G' :  19,
    ...
}


In [6]:
print("{} --- char2idx ---> {}".format(repr(text[:13]), text_as_int[:13]))

'First Citizen' --- char2idx ---> [18 47 56 57 58  1 15 47 58 47 64 43 52]


## Prediction tast
Given a character, or sequence of characters, what is the most probable next character?
## Create training examples and targets
For each input sequence, corresponding targets contain the same length of text, except shifted one character to the right.

In [7]:
# Convert text vector into a stream of character indices
# Maximum length sentence for a single input in characters
seq_len = 100
ex_per_epoch = len(text) // seq_len
print(ex_per_epoch)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(13):
    print(idx2char[i.numpy()])

11153
F
i
r
s
t
 
C
i
t
i
z
e
n


In [8]:
# use batch method to convert individual characters to sequences of 'seq_len'
sequences = char_dataset.batch(seq_len + 1, drop_remainder=True)

for item in sequences.take(5):
    print("***\n{}\n***\n".format(''.join(idx2char[item])))

***
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
***

***
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k
***

***
now Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us ki
***

***
ll him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be d
***

***
one: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citi
***



In [9]:
# Duplicate and shift each sequence to form the input + target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for y, t in dataset.take(1):
    print("y: {}".format(repr(''.join(idx2char[y.numpy()]))))
    print("t: {}".format(repr(''.join(idx2char[t.numpy()]))))

y: 'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
t: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [10]:
# Observe how the data will be processed by the model
first_5 = [(y[:5], t[:5]) for y, t in dataset.take(1)]
for i, (y, t) in enumerate(zip(first_5[0][0], first_5[0][1])):
    print("Step {:4d}".format(i))
    print("    input: {} ({:s})".format(y, repr(idx2char[y])))
    print("    expected output: {} ({:s})".format(t, repr(idx2char[t])))

Step    0
    input: 18 ('F')
    expected output: 47 ('i')
Step    1
    input: 47 ('i')
    expected output: 56 ('r')
Step    2
    input: 56 ('r')
    expected output: 57 ('s')
Step    3
    input: 57 ('s')
    expected output: 58 ('t')
Step    4
    input: 58 ('t')
    expected output: 1 (' ')


## Create training batches
Shuffle the data and pack it into batches

In [11]:
# batch size
BATCH_SIZE = 64
steps_per_epoch = ex_per_epoch // BATCH_SIZE
print(steps_per_epoch)

# TF doesn't shuffle the entire sequence, just the amount that fits in the buffer
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

174
<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>


# Build the model

In [12]:
# Length of vocab in chars
VOCAB_SIZE = len(vocab)

# Embedding dimension
EMBEDDING_DIM = 256

# Number of RNN units
RNN_UNITS = 1024

if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(
        tf.keras.layers.GRU, recurrent_activation="sigmoid"
    )

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(
            vocab_size, embedding_dim, batch_input_shape=(batch_size, None)
        ),
        rnn(
            rnn_units, return_sequences=True, recurrent_initializer="glorot_uniform",
            stateful=True
        ),
        tf.keras.layers.Dense(vocab_size, activation="softmax")
    ])
    return model

In [14]:
model = build_model(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNITS,
    batch_size=BATCH_SIZE
)

# Try the model

In [15]:
# Check shape of the output
for y_ex_batch, t_ex_batch in dataset.take(1):
    y_ex_batch_pred = model(y_ex_batch)
    print("(batch size, sequence length, vocab size): {}".format(y_ex_batch_pred.shape))

(batch size, sequence length, vocab size): (64, 100, 65)


In [16]:
# Sequence length of input is 100, but model can be run on inputs of any length
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3935232   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,018,497
Trainable params: 4,018,497
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Sample from the output distribution to get character indices
# These predictions will basically be random since the model is untrained
for y_ex_batch, t_ex_batch in dataset.take(1):
    y_ex_batch_pred = model(y_ex_batch)
    sampled_indices = tf.multinomial(y_ex_batch_pred[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
    print("Sampled indices:\n{}".format(sampled_indices))
    print("Input:\n{}".format(repr(''.join(idx2char[y_ex_batch[0]]))))
    print("\nNext char predictions:\n{}".format(repr(''.join(idx2char[sampled_indices]))))

Sampled indices:
[34 28 26  5 19 39 26 53 44 55  6 29 48 47 62 40 60 29 25 10 49  2  9 26
 48  9 30 50 50 24 48 50  9 39 49 11 35 57 15 33  1 13 61  3 44 57 19 13
 25 16 49 46  5  4 51 19 52 10 61 12 46 14  6  0 49 30 61 26 34 22 13 27
 62 33 56 62 61 31  5 45  8 46 42 57 38 28 26  4 62 61 22 64 10 55 38 13
 39 11 59 31]
Input:
'hat is at mercy? Five times, Marcius,\nI have fought with thee: so often hast thou beat me,\nAnd would'

Next char predictions:
"VPN'GaNofq,QjixbvQM:k!3Nj3RllLjl3ak;WsCU Aw$fsGAMDkh'&mGn:w?hB,\nkRwNVJAOxUrxwS'g.hdsZPN&xwJz:qZAa;uS"


# Train the model

## Attach an optimizer and loss function

In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits)

In [19]:
for y_ex_batch, t_ex_batch in dataset.take(1):
    ex_batch_pred = model(y_ex_batch)
    ex_batch_loss = loss(t_ex_batch, ex_batch_pred)
    print("Prediction shape: {} # (batch size, sequence length, vocab size)".format(ex_batch_pred.shape))
    print("Scalar loss: {}".format(ex_batch_loss.numpy().mean()))

Prediction shape: (64, 100, 65) # (batch size, sequence length, vocab size)
Scalar loss: 4.173377513885498
